In [ ]:
import keras
from keras.layers import Input, ReLU, BatchNormalization,Dense, Dropout
from keras.models import Model
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
from keras import regularizers

In [35]:
import pandas as pd
import random
#trainx=pd.read_csv('../data/x_1730.csv')
#trainy=pd.read_csv('../data/y_1730.csv')
trainx=pd.read_csv('../data/x_600_2_JN.csv')
trainy=pd.read_csv('../data/y_600_2_JN.csv')

In [36]:
index =list(trainx.index)
random.Random(0).shuffle(index)
num=int(len(index)/5*4)
x_train,y_train=trainx.iloc[index[:num],:].values,trainy.iloc[index[:num],:].values
x_test,y_test=trainx.iloc[index[num:],:].values,trainy.iloc[index[num:],:].values
data_X=np.vstack([x_train,x_test])

In [ ]:
K.clear_session()
N=400
H1=200
H2=48
le=0
drop_rate=0.1
input_img=Input(shape=(N,))
# 编码层
encoded=Dense(H1,activation='relu',activity_regularizer=regularizers.l1(le),kernel_regularizer=regularizers.l2(0.01))(input_img)
encoded=BatchNormalization()(encoded)
encoded=Dropout(drop_rate)(encoded)
encoded=Dense(H2,activation='relu',activity_regularizer=regularizers.l1(le),kernel_regularizer=regularizers.l2(0.01))(encoded)
encoder_output=BatchNormalization()(encoded)
encoder_output=Dropout(drop_rate)(encoder_output)
LR=Dense(4,activation='softmax',name='LR')(encoder_output)

# 解码层
decoded=Dense(H2,activation='relu',name='decoded_hidden3')(encoder_output)
decoded=Dense(H1,activation='relu',name='decoded_hidden1')(decoded)
decoded=Dense(N,activation='tanh',name='decoded_output')(decoded)

# 构建自编码模型
autoencoder=Model(inputs=input_img,outputs=decoded)

# complile autoencoder 设置自编码的优化参数
autoencoder.compile(optimizer='adam',loss='mse')
# train
hist=autoencoder.fit(data_X,data_X,epochs=15,batch_size=128,shuffle=True)

In [ ]:
encoder=Model(inputs=input_img,outputs=LR)
encoder.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
encoder.fit(x_train,y_train,epochs=100,batch_size=248,shuffle=True)
score=encoder.evaluate(x_test,y_test)
print(score)

In [3]:
list(range(10,-1,-1))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]